### Load cleaned data
Let's load the cleaned dataset so we can engineer features on a consistent base.  
From this, we’ll verify column types and row counts before making changes.

In [1]:
import pandas as pd

df = pd.read_csv('../data/clean.csv')
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


### Add a small noise feature (sanity check / leakage guard)
Let’s add a `random_noise` column (N(0,1)) to catch accidental leakage or overly flexible models.  
From that, we expect **no** real signal; if it shows up as important, something’s fishy.

In [2]:
import numpy as np 

np.random.seed(42)
df['random_noise'] = np.random.normal(loc=0, scale=1, size=len(df))
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X16,X17,X18,X19,X20,X21,X22,X23,Y,random_noise
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,689,0,0,0,0,1,0.496714
1,120000,2,2,2,26,-1,2,0,0,0,...,3455,3261,0,1000,1000,1000,0,2000,1,-0.138264
2,90000,2,2,2,34,0,0,0,0,0,...,14948,15549,1518,1500,1000,1000,1000,5000,0,0.647689
3,50000,2,2,1,37,0,0,0,0,0,...,28959,29547,2000,2019,1200,1100,1069,1000,0,1.523030
4,50000,1,2,1,57,-1,0,-1,0,0,...,19146,19131,2000,36681,10000,9000,689,679,0,-0.234153


### X1: Transform & encodings
Let’s engineer multiple representations of credit limit (`X1`):
- `X1_log`: log1p transform to tame right skew.  
- `X1_ordinal`: ordinal-encode (compact `uint*` dtype based on cardinality).  
- `X1_bin`: quantile bins (KBinsDiscretizer) to expose non-linear splits.

From that, we get three complementary views of capacity: scale-stabilized, encoded, and binned.


In [3]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder, KBinsDiscretizer

# Log transform
df['X1_log'] = np.log1p(df['X1']).astype('float32')

# Ordinal encode
ordinal_encoder = OrdinalEncoder()
df['X1_ordinal'] = ordinal_encoder.fit_transform(df[['X1']])
n_unique = df['X1_ordinal'].nunique()
if n_unique <= 255:
    df['X1_ordinal'] = df['X1_ordinal'].astype('uint8')
elif n_unique <= 65535:
    df['X1_ordinal'] = df['X1_ordinal'].astype('uint16')
else:
    df['X1_ordinal'] = df['X1_ordinal'].astype('uint32')

# Quantile bin
binner = KBinsDiscretizer(n_bins=4, encode='ordinal', quantile_method='averaged_inverted_cdf', strategy='quantile')
df['X1_bin'] = binner.fit_transform(df[['X1']]).astype('uint8')

df['X1'] = df['X1'].astype('int32')


### X2: One-hot encode gender
Let’s one-hot encode `X2` (drop_first=True) and cast to `uint8`.  
From that, we avoid ordinality assumptions and keep storage tiny.


In [4]:
df['X2'] = df['X2'].astype('category')
gender_dummies = pd.get_dummies(df['X2'], prefix='X2', drop_first=True).astype('uint8')
df = pd.concat([df.drop(columns=['X2']), gender_dummies], axis=1)

df.head()

,X1,X3,X4,X5,X6,X7,X8,X9,X10,X11,...,X20,X21,X22,X23,Y,random_noise,X1_log,X1_ordinal,X1_bin,X2_2
0,20000,2,1,24,2,2,-1,-1,-2,-2,...,0,0,0,0,1,0.496714,9.903538,2,0,1
1,120000,2,2,26,-1,2,0,0,0,2,...,1000,1000,0,2000,1,-0.138264,11.695255,12,1,1
2,90000,2,2,34,0,0,0,0,0,0,...,1000,1000,1000,5000,0,0.647689,11.407576,9,1,1
3,50000,2,1,37,0,0,0,0,0,0,...,1200,1100,1069,1000,0,1.523030,10.819798,5,1,1
4,50000,2,1,57,-1,0,-1,0,0,0,...,10000,9000,689,679,0,-0.234153,10.819798,5,1,0


### X3: Group education levels
Let’s map `X3` into fewer, statistically coherent buckets (from EDA findings):  
- 2/3 → **Group A**  
- 4/5/6 → **Group B**  
- 1 → **Group D (graduate)**  
- else (0/unknown) → **Group C (inconclusive)**

From that, `X3_grouped` (uint8) reflects signal while reducing sparsity.


In [5]:
def map_education(x):
    if x in [2, 3]:
        return 0  # Group A
    elif x in [4, 5, 6]:
        return 1  # Group B
    elif x == 1:
        return 2  # Group D (graduate school)
    else:  # x == 0 or unknown
        return 3  # Group C (inconclusive)

df['X3_grouped'] = df['X3'].apply(map_education).astype('uint8')


### X4: Group + one-hot marital status
Let’s collapse `X4` into labels and one-hot encode:  
- {1,3} → `Married_or_Other`  
- {2} → `Single`  
- else → `Unknown`

From that, we replace the original with compact `uint8` dummies aligned to our EDA.


In [6]:
# Apply grouping (convert to string labels)
def map_marital_status(x):
    if x in [1, 3]:
        return 'Married_or_Other'
    elif x == 2:
        return 'Single'
    else:
        return 'Unknown'

# Apply mapping
df['X4_grouped'] = df['X4'].apply(map_marital_status)

# One-hot encode with numeric dtype (uint8)
X4_dummies = pd.get_dummies(df['X4_grouped'], prefix='X4', dtype='uint8')

# Drop original and grouped columns, then concatenate encoded columns
df = pd.concat([df.drop(columns=['X4', 'X4_grouped']), X4_dummies], axis=1)


## X5

In [7]:
# Do nothing 

### X6–X11: Payment-status behavior features
Let’s summarize six months of repayment status (recent → older):

**Delinquency profile**
- `X6_X11_delinquency_count`: months with status ≥ 1  
- `X6_X11_max_delay`: worst delay  
- `X6_X11_avg_delay`: mean delay  
- `X6_X11_severe_delinquency_count`: months with status ≥ 3

**Positive/no-use behavior**
- `X6_X11_paid_in_full_count` (status = −1)  
- `X6_X11_no_use_count` (status = −2)  
- `X6_X11_revolving_count` (status = 0)

**Recency flags**
- `X6_X11_last_paid_in_full`: months since last full pay  
- `X6_X7_recent_delinquent`: delinquent in the most recent 1–2 months?

**Weighted score**
- `X6_X11_weighted_score`: heavier weight on recent months.

From that, we turn monthly statuses into stable, model-friendly signals that emphasize recency.


In [8]:
for col in ['X6', 'X7', 'X8', 'X9', 'X10', 'X11']:
    df[col] = df[col].astype('int8')  # range from -2 to 9 fits in int8

# Count of months with any delinquency (1 or more)
df['X6_X11_delinquency_count'] = (df[['X6','X7','X8','X9','X10','X11']] >= 1).sum(axis=1).astype('uint8')

# Max months of delay (worst delinquency)
df['X6_X11_max_delay'] = df[['X6','X7','X8','X9','X10','X11']].max(axis=1).astype('int8')

# Mean delay across all months
df['X6_X11_avg_delay'] = df[['X6','X7','X8','X9','X10','X11']].mean(axis=1).astype('float32')

# Count of severe delinquencies (3 months or more)
df['X6_X11_severe_delinquency_count'] = (df[['X6','X7','X8','X9','X10','X11']] >= 3).sum(axis=1).astype('uint8')

# Months with full payment (-1)
df['X6_X11_paid_in_full_count'] = (df[['X6','X7','X8','X9','X10','X11']] == -1).sum(axis=1).astype('uint8')

# Months with no card use (-2)
df['X6_X11_no_use_count'] = (df[['X6','X7','X8','X9','X10','X11']] == -2).sum(axis=1).astype('uint8')

# Months revolving balance but not delinquent (== 0)
df['X6_X11_revolving_count'] = (df[['X6','X7','X8','X9','X10','X11']] == 0).sum(axis=1).astype('uint8')

# How many months since last full payment (-1)
df['X6_X11_last_paid_in_full'] = df[['X6','X7','X8','X9','X10','X11']].iloc[:, ::-1].eq(-1).idxmax(axis=1).str.extract(r'X(\d+)').astype(float).sub(6, axis=0)[0].fillna(6).astype('uint8')

# Has the user been delinquent in the most recent 2 months?
df['X6_X7_recent_delinquent'] = ((df['X6'] >= 1) | (df['X7'] >= 1)).astype('uint8')

# Weighted risk score (optional — can tune weights)
weights = np.array([5, 4, 3, 2, 1, 1])  # Heavier weight on recent
df['X6_X11_weighted_score'] = (df[['X6','X7','X8','X9','X10','X11']].clip(lower=0).values * weights).sum(axis=1).astype('int16')


### X12–X17: Bill amounts (Apr→Sep) — scale & trends
Let’s type-tighten to `int32` and create signed log versions for robustness to zeros/negatives:  
`{col}_log_signed = sign(col) * log1p(abs(col))`.

Then we build:
- `bill_growth` = recent (X12) − oldest (X17)  
- `bill_trend` = linear slope across X17→X12  
- `bill_std` = dispersion over six months  
- `bill_range` = max − min  
- `bill_weighted_mean` = recency-weighted average (more weight on recent)

From that, we capture level, volatility, and momentum of outstanding bills.


In [9]:
for col in ['X12', 'X13', 'X14', 'X15', 'X16', 'X17']:
    df[col] = df[col].astype('int32')
  
for col in ['X12', 'X13', 'X14', 'X15', 'X16', 'X17']:
    orig = df[col].astype('float32')  # ensure float for sign ops
    abs_logged = np.log1p(np.abs(orig))
    df[f'{col}_log_signed'] = (abs_logged * np.sign(orig)).astype('float32')

# Monthly bill growth (recent - oldest)
df['bill_growth'] = (df['X12'] - df['X17']).astype('int32')

# Slope / trend (approximate with linear fit)
df['bill_trend'] = df[['X17','X16','X15','X14','X13','X12']].apply(
    lambda row: np.polyfit(range(6), row.values, 1)[0], axis=1
).astype('float32')

# Standard deviation
df['bill_std'] = df[['X12','X13','X14','X15','X16','X17']].std(axis=1).astype('float32')

# Range
df['bill_range'] = (df[['X12','X13','X14','X15','X16','X17']].max(axis=1) - df[['X12','X13','X14','X15','X16','X17']].min(axis=1)).astype('int32')

# Weighted average: emphasize more recent months
weights = np.array([1, 2, 3, 4, 5, 6])  # oldest to newest
df['bill_weighted_mean'] = (df[['X17','X16','X15','X14','X13','X12']].values * weights).sum(axis=1) / weights.sum()
df['bill_weighted_mean'] = df['bill_weighted_mean'].astype('float32')


### X18–X23: Payment amounts (Apr→Sep) — totals & consistency
Let’s add signed-log versions, cast to `int32`, and summarize:

- `payment_total`, `payment_avg`, `payment_std`, `payment_max`, `payment_min`  
- `payment_growth` = most recent (X18) − oldest (X23)  
- `payment_trend` = linear slope across X23→X18  
- `payment_zero_count` = how many months had zero payment  
- `payment_all_zero` = binary flag (all six months zero)  
- `payment_low_count` = months with very low payments (e.g., < 1000 NT)

From that, we model repayment capacity, regularity, and stress indicators.


In [10]:
payment_cols = ['X18', 'X19', 'X20', 'X21', 'X22', 'X23']

for col in payment_cols:
    orig = df[col].astype('float32')
    abs_logged = np.log1p(np.abs(orig))
    df[f'{col}_log_signed'] = (abs_logged * np.sign(orig)).astype('float32')
    
for col in payment_cols:
    df[col] = df[col].astype('int32')

# Total paid over 6 months
df['payment_total'] = df[payment_cols].sum(axis=1).astype('int32')

# Mean payment
df['payment_avg'] = df[payment_cols].mean(axis=1).astype('float32')

# Standard deviation
df['payment_std'] = df[payment_cols].std(axis=1).astype('float32')

# Max and min payments
df['payment_max'] = df[payment_cols].max(axis=1).astype('int32')
df['payment_min'] = df[payment_cols].min(axis=1).astype('int32')

# Change from oldest (X23) to most recent (X18)
df['payment_growth'] = (df['X18'] - df['X23']).astype('int32')

# Linear trend (slope)
df['payment_trend'] = df[['X23','X22','X21','X20','X19','X18']].apply(
    lambda row: np.polyfit(range(6), row.values, 1)[0], axis=1
).astype('float32')

# Count of zero payments
df['payment_zero_count'] = (df[payment_cols] == 0).sum(axis=1).astype('uint8')

# Binary flag: All 6 months were zero
df['payment_all_zero'] = (df['payment_zero_count'] == 6).astype('uint8')

# Number of months with "very low" payments (e.g., < 1000 NT)
df['payment_low_count'] = (df[payment_cols] < 1000).sum(axis=1).astype('uint8')



### Final schema check
Let’s re-inspect `df.info()` to confirm feature counts, dtypes, and memory footprint.  
From that, we ensure all engineered features are present and compact (`uint8`/`int32`/`float32` where possible).


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29965 entries, 0 to 29964
Data columns (total 68 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   X1                               29965 non-null  int32  
 1   X3                               29965 non-null  int64  
 2   X5                               29965 non-null  int64  
 3   X6                               29965 non-null  int8   
 4   X7                               29965 non-null  int8   
 5   X8                               29965 non-null  int8   
 6   X9                               29965 non-null  int8   
 7   X10                              29965 non-null  int8   
 8   X11                              29965 non-null  int8   
 9   X12                              29965 non-null  int32  
 10  X13                              29965 non-null  int32  
 11  X14                              29965 non-null  int32  
 12  X15               

### Save engineered dataset
Let’s persist the engineered features for downstream modeling and splits: `../data/engineered.csv`.  
From that, training pipelines can consume a consistent, versioned artifact.


In [12]:
df.to_csv('../data/engineered.csv')